# Content-Based Recommender Systems 

# Importing the Datasets (Slim)

In [1]:
import pandas as pd
import ast

# Load the ratings data
ratings_df = pd.read_csv('../dataset/XWines_Slim_150K_ratings.csv')

# Load the wines data
wines_df = pd.read_csv('../dataset/XWines_Slim_1K_wines.csv')

/var/folders/9n/hfw66rw97sl45x98mmd_vs4r0000gn/T/ipykernel_44092/3978127779.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df = pd.read_csv('../dataset/XWines_Slim_150K_ratings.csv')


# Data Pre-Processing

## Ratings Table

In [2]:
ratings_df.head()

,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38


In [3]:
# Dropping the Date column from ratings_df
ratings_df.drop(columns=['Date'], inplace=True)
ratings_df.head()

,RatingID,UserID,WineID,Vintage,Rating
0,143,1356810,103471,1950,4.5
1,199,1173759,111415,1951,5.0
2,348,1164877,111395,1952,5.0
3,374,1207665,111433,1953,5.0
4,834,1075841,111431,1955,5.0


## Wines Table

In [4]:
# Taking a Peak at wines_df and seeing which columns have repeated info
wines_df.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10012,Dal Pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [5]:
# Go through every column in wines_df and print the number of distinct values in a descriptve way
for column in wines_df.columns:
    print(f"{column}: {wines_df[column].nunique()}") 

WineID: 1007
WineName: 803
Type: 6
Elaborate: 13
Grapes: 304
Harmonize: 205
ABV: 76
Body: 5
Acidity: 3
Code: 31
Country: 31
RegionID: 324
RegionName: 324
WineryID: 792
WineryName: 791
Website: 689
Vintages: 967


In [6]:
# Dropping some of the Columns in the wines_df
# Some are irrelevant for analysis (like the website) and some will just be repeated info (like the Code and Country)
columns_to_drop_wines = ['Website', 'Code', 'RegionID', 'WineryID']
wines_df.drop(columns=columns_to_drop_wines, inplace=True)
wines_df.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Country,RegionName,WineryName,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Casa Perini,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,Brazil,Serra Gaúcha,Casa Perini,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,Brazil,Serra Gaúcha,Castellamare,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,Brazil,Serra Gaúcha,Aurora,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Dal Pizzol,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [7]:
import ast

# List of columns to parse
list_columns = ['Grapes', 'Harmonize', 'Vintages']

# Function to parse the stringified lists
def parse_list(column, is_vintages=False):
    def parse_item(item):
        try:
            # Safely evaluate the string to a Python object
            parsed = ast.literal_eval(item)
            # Ensure the parsed object is a list
            if not isinstance(parsed, list):
                return []
            # For 'Vintages', filter out non-numeric entries and convert to integers
            if is_vintages:
                parsed = [
                    int(year) for year in parsed
                    if (isinstance(year, int)) or (isinstance(year, str) and year.isdigit())
                ]
            return parsed
        except (ValueError, SyntaxError):
            return []
    return column.apply(parse_item)

# Apply the function to each column
for col in list_columns:
    if col == 'Vintages':
        wines_df[col] = parse_list(wines_df[col], is_vintages=True)
    else:
        wines_df[col] = parse_list(wines_df[col])


In [10]:
type(wines_df.at[0, 'Grapes']) 

list

In [11]:
type(wines_df.at[0, 'Harmonize']) 

list

In [12]:
type(wines_df.at[0, 'Vintages']) 

list

In [13]:
wines_df.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Country,RegionName,WineryName,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,[Muscat/Moscato],"[Pork, Rich Fish, Shellfish]",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Casa Perini,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,[Ancellotta],"[Beef, Barbecue, Codfish, Pasta, Pizza, Cheese]",12.0,Medium-bodied,Medium,Brazil,Serra Gaúcha,Casa Perini,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,[Cabernet Sauvignon],"[Beef, Lamb, Poultry]",12.0,Full-bodied,High,Brazil,Serra Gaúcha,Castellamare,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"[Cabernet Sauvignon, Merlot]","[Beef, Lamb, Game Meat, Poultry]",11.0,Full-bodied,Medium,Brazil,Serra Gaúcha,Aurora,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,[Muscat/Moscato Bianco],"[Pork, Rich Fish, Shellfish]",7.5,Medium-bodied,High,Brazil,Serra Gaúcha,Dal Pizzol,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


# Simple Content Based Classifier (Based on Similarity alone)

In [40]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import jaccard
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import ndcg_score


In [41]:
# Handle missing values in wines_df
categorical_features = ['Type', 'Country', 'Body', 'Acidity']
wines_df[categorical_features] = wines_df[categorical_features].fillna('Unknown')
wines_df['ABV'] = wines_df['ABV'].fillna(wines_df['ABV'].mean())

# Ensure 'WineID' is of type string
wines_df['WineID'] = wines_df['WineID'].astype(str)


In [42]:
# Drop rows with missing values in ratings_df
ratings_df = ratings_df.dropna(subset=['UserID', 'WineID', 'Rating'])

# Ensure 'UserID' and 'WineID' are of type string
ratings_df['UserID'] = ratings_df['UserID'].astype(str)
ratings_df['WineID'] = ratings_df['WineID'].astype(str)


In [43]:
# Step 3: Define the ContentBasedRecommender Class
class ContentBasedRecommender(BaseEstimator):
    def __init__(self, n_recommendations=10, feature_weights=None, distance_metric='cosine'):
        self.n_recommendations = n_recommendations
        self.feature_weights = feature_weights
        self.distance_metric = distance_metric

    def fit(self, item_features_df, user_ratings_df):
        self.item_features_df = item_features_df.copy()
        self.user_ratings_df = user_ratings_df.copy()

        # Ensure 'WineID' is of the same type
        self.item_features_df['WineID'] = self.item_features_df['WineID'].astype(str)
        self.user_ratings_df['WineID'] = self.user_ratings_df['WineID'].astype(str)

        # Set 'WineID' as the index for item features
        self.item_features_df = self.item_features_df.set_index('WineID')

        # Preprocess item features
        self._preprocess_item_features()

        # Build user profiles
        self._build_user_profiles()

        # Compute similarity matrix
        self._compute_similarity()

        return self

    def _preprocess_item_features(self):
        # Handle missing values
        categorical_features = ['Type', 'Country', 'Body', 'Acidity']
        self.item_features_df[categorical_features] = self.item_features_df[categorical_features].fillna('Unknown')
        self.item_features_df['ABV'] = self.item_features_df['ABV'].fillna(self.item_features_df['ABV'].mean())

        # One-hot encode categorical features
        self.encoded_features = pd.get_dummies(self.item_features_df[categorical_features])

        # Normalize numerical features
        scaler = MinMaxScaler()
        abv_scaled = scaler.fit_transform(self.item_features_df[['ABV']])
        abv_scaled_df = pd.DataFrame(abv_scaled, columns=['ABV_Scaled'], index=self.item_features_df.index)

        # Combine all features
        self.item_feature_matrix = pd.concat([self.encoded_features, abv_scaled_df], axis=1)

        # Apply feature weights if provided
        if self.feature_weights:
            for feature, weight in self.feature_weights.items():
                if feature in self.item_feature_matrix.columns:
                    self.item_feature_matrix[feature] *= weight

        # Check for NaN values in item_feature_matrix
        if self.item_feature_matrix.isnull().values.any():
            print("NaN values found in item_feature_matrix:")
            print(self.item_feature_matrix.isnull().sum()[self.item_feature_matrix.isnull().sum() > 0])
            # Fill NaN values with zeros
            self.item_feature_matrix = self.item_feature_matrix.fillna(0)

    def _build_user_profiles(self):
        user_groups = self.user_ratings_df.groupby('UserID')
        user_profiles_list = []

        for user_id, group in user_groups:
            num_ratings = len(group)
            num_top_items = max(1, int(np.ceil(num_ratings * 0.25)))
            user_ratings_sorted = group.sort_values('Rating', ascending=False)
            top_items = user_ratings_sorted.head(num_top_items)

            # Merge with item features
            top_items = top_items.merge(
                self.item_feature_matrix, left_on='WineID', right_index=True
            )

            if top_items.empty:
                print(f"Warning: User {user_id} has no items after merging with item features.")
                user_profile = pd.Series(0, index=self.item_feature_matrix.columns)
            else:
                user_profile = top_items[self.item_feature_matrix.columns].mean()

            user_profiles_list.append({'UserID': user_id, **user_profile})

        # Create the user profiles DataFrame
        self.user_profiles = pd.DataFrame(user_profiles_list).set_index('UserID')

        # Fill NaN values in user_profiles
        if self.user_profiles.isnull().values.any():
            print("NaN values found in user_profiles:")
            print(self.user_profiles.isnull().sum()[self.user_profiles.isnull().sum() > 0])
            # Fill NaN values with zeros
            self.user_profiles = self.user_profiles.fillna(0)

        # Normalize user profiles
        self.user_profiles = self.user_profiles.apply(
            lambda x: x / np.linalg.norm(x) if np.linalg.norm(x) != 0 else x,
            axis=1
        )

    def _compute_similarity(self):
        if self.distance_metric == 'cosine':
            self.similarity_matrix = cosine_similarity(self.user_profiles, self.item_feature_matrix)
        elif self.distance_metric == 'euclidean':
            distances = euclidean_distances(self.user_profiles, self.item_feature_matrix)
            self.similarity_matrix = 1 / (1 + distances)
        elif self.distance_metric == 'manhattan':
            distances = manhattan_distances(self.user_profiles, self.item_feature_matrix)
            self.similarity_matrix = 1 / (1 + distances)
        elif self.distance_metric == 'jaccard':
            from sklearn.metrics import pairwise_distances
            user_profiles_binary = self.user_profiles.applymap(lambda x: 1 if x > 0 else 0)
            item_features_binary = self.item_feature_matrix.applymap(lambda x: 1 if x > 0 else 0)
            self.similarity_matrix = 1 - pairwise_distances(user_profiles_binary, item_features_binary, metric='jaccard')
        else:
            raise ValueError("Unsupported distance metric. Choose from 'cosine', 'euclidean', 'manhattan', 'jaccard'.")

        # Map user IDs and item indices
        self.user_id_to_index = {user_id: idx for idx, user_id in enumerate(self.user_profiles.index)}
        self.index_to_item_id = {idx: item_id for idx, item_id in enumerate(self.item_feature_matrix.index)}

    def predict(self, user_ids):
        """
        Generate recommendations for a list of user IDs.

        Parameters:
        - user_ids: List of user IDs.

        Returns:
        - recommendations: Dictionary mapping user IDs to lists of recommended WineIDs.
        """
        recommendations = {}
        for user_id in user_ids:
            if user_id in self.user_id_to_index:
                user_idx = self.user_id_to_index[user_id]
                sim_scores = self.similarity_matrix[user_idx]
                # Exclude items already rated by the user
                rated_items = self.user_ratings_df[self.user_ratings_df['UserID'] == user_id]['WineID'].tolist()
                # Create a mapping of item indices to IDs
                item_indices_to_ids = {idx: item_id for idx, item_id in enumerate(self.item_feature_matrix.index)}
                # Filter out rated items
                unrated_items = [(idx, sim_scores[idx]) for idx in range(len(sim_scores)) if item_indices_to_ids[idx] not in rated_items]
                # Get top N recommendations
                top_items = sorted(unrated_items, key=lambda x: x[1], reverse=True)[:self.n_recommendations]
                recommended_item_ids = [item_indices_to_ids[idx] for idx, score in top_items]
                recommendations[user_id] = recommended_item_ids
            else:
                # User not found in training data
                recommendations[user_id] = []
        return recommendations

    def evaluate(self, user_ids, ground_truth, top_n=10):
        """
        Evaluate the model using various metrics at N.

        Parameters:
        - user_ids: List of user IDs to evaluate.
        - ground_truth: DataFrame containing ground truth ratings (UserID, WineID, Rating).
        - top_n: Number of top recommendations to consider for evaluation.

        Returns:
        - metrics: Dictionary containing evaluation metrics.
        """
        recommendations = self.predict(user_ids)
        total_users = len(user_ids)
        precision_sum = 0.0
        recall_sum = 0.0
        f1_sum = 0.0
        map_sum = 0.0
        mrr_sum = 0.0
        ndcg_sum = 0.0

        for user_id in user_ids:
            recommended_items = recommendations.get(user_id, [])[:top_n]
            relevant_items = ground_truth[ground_truth['UserID'] == user_id]['WineID'].tolist()
            hits = [1 if item in relevant_items else 0 for item in recommended_items]
            num_relevant_items = len(relevant_items)

            # Precision@N
            precision = sum(hits) / top_n if top_n > 0 else 0
            precision_sum += precision

            # Recall@N
            recall = sum(hits) / num_relevant_items if num_relevant_items > 0 else 0
            recall_sum += recall

            # F1@N
            if (precision + recall) > 0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0
            f1_sum += f1

            # MAP@N
            map_score = 0.0
            hit_count = 0
            for idx, hit in enumerate(hits):
                if hit:
                    hit_count += 1
                    map_score += hit_count / (idx + 1)
            map_score = map_score / min(num_relevant_items, top_n) if num_relevant_items > 0 else 0
            map_sum += map_score

            # MRR@N
            mrr = 0.0
            for idx, hit in enumerate(hits):
                if hit:
                    mrr = 1 / (idx + 1)
                    break
            mrr_sum += mrr

            # NDCG@N
            if len(hits) > 0 and sum(hits) > 0:
                ndcg = ndcg_at_k(hits, top_n)
            else:
                ndcg = 0.0
            ndcg_sum += ndcg

        # Average metrics
        metrics = {
            'Precision@{}'.format(top_n): precision_sum / total_users,
            'Recall@{}'.format(top_n): recall_sum / total_users,
            'F1@{}'.format(top_n): f1_sum / total_users,
            'MAP@{}'.format(top_n): map_sum / total_users,
            'MRR@{}'.format(top_n): mrr_sum / total_users,
            'NDCG@{}'.format(top_n): ndcg_sum / total_users
        }
        return metrics

In [44]:
# Custom DCG and NDCG functions
def dcg_at_k(relevance_scores, k):
    relevance_scores = np.array(relevance_scores)[:k]
    if relevance_scores.size:
        discounts = np.log2(np.arange(2, relevance_scores.size + 2))
        return np.sum(relevance_scores / discounts)
    return 0.0

def ndcg_at_k(relevance_scores, k):
    dcg_max = dcg_at_k(sorted(relevance_scores, reverse=True), k)
    if dcg_max == 0:
        return 0.0
    return dcg_at_k(relevance_scores, k) / dcg_max

In [45]:
# Step 4: Set Up K-Fold Cross-Validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
user_ids = ratings_df['UserID'].unique()

In [46]:
# Step 5: Train and Evaluate the Model
metrics_list = []

for fold, (train_index, test_index) in enumerate(kf.split(user_ids), 1):
    print(f"\nProcessing Fold {fold}/{n_splits}")
    
    user_ids_train = user_ids[train_index]
    user_ids_test = user_ids[test_index]
    
    train_ratings = ratings_df[ratings_df['UserID'].isin(user_ids_train)]
    test_ratings = ratings_df[ratings_df['UserID'].isin(user_ids_test)]
    
    recommender = ContentBasedRecommender(
        n_recommendations=10,
        feature_weights=None,
        distance_metric='cosine'
    )
    
    recommender.fit(item_features_df=wines_df, user_ratings_df=train_ratings)
    
    metrics = recommender.evaluate(user_ids=user_ids_test, ground_truth=test_ratings, top_n=10)
    metrics['Fold'] = fold
    metrics_list.append(metrics)
    
    print(f"Metrics for Fold {fold}:")
    for metric_name, value in metrics.items():
        if metric_name != 'Fold':
            print(f"{metric_name}: {value:.4f}")
    print('-' * 40)



Processing Fold 1/5
Metrics for Fold 1:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
----------------------------------------

Processing Fold 2/5
Metrics for Fold 2:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
----------------------------------------

Processing Fold 3/5
Metrics for Fold 3:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
----------------------------------------

Processing Fold 4/5
Metrics for Fold 4:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
----------------------------------------

Processing Fold 5/5
Metrics for Fold 5:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
----------------------------------------


In [47]:
# Step 6: Aggregate and Display the Results
metrics_df = pd.DataFrame(metrics_list)
average_metrics = metrics_df.mean(numeric_only=True)

print("\nAverage Metrics Across All Folds:")
for metric_name, value in average_metrics.items():
    print(f"{metric_name}: {value:.4f}")


Average Metrics Across All Folds:
Precision@10: 0.0000
Recall@10: 0.0000
F1@10: 0.0000
MAP@10: 0.0000
MRR@10: 0.0000
NDCG@10: 0.0000
Fold: 3.0000


In [48]:
# Step 1: Import Necessary Libraries
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.model_selection import KFold

# Preprocess wines_df
categorical_features = ['Type', 'Country', 'Body', 'Acidity']
wines_df[categorical_features] = wines_df[categorical_features].fillna('Unknown')
wines_df['ABV'] = wines_df['ABV'].fillna(wines_df['ABV'].mean())
wines_df['WineID'] = wines_df['WineID'].astype(str)

# Preprocess ratings_df
ratings_df = ratings_df.dropna(subset=['UserID', 'WineID', 'Rating'])
ratings_df['UserID'] = ratings_df['UserID'].astype(str)
ratings_df['WineID'] = ratings_df['WineID'].astype(str)

# Step 3: Define the ContentBasedRecommender Class
# (Include the class definition with custom NDCG functions from before)

# Custom DCG and NDCG functions
def dcg_at_k(relevance_scores, k):
    relevance_scores = np.array(relevance_scores)[:k]
    if relevance_scores.size:
        discounts = np.log2(np.arange(2, relevance_scores.size + 2))
        return np.sum(relevance_scores / discounts)
    return 0.0

def ndcg_at_k(relevance_scores, k):
    dcg_max = dcg_at_k(sorted(relevance_scores, reverse=True), k)
    if dcg_max == 0:
        return 0.0
    return dcg_at_k(relevance_scores, k) / dcg_max

# Step 4: Set Up K-Fold Cross-Validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
ratings_indices = np.arange(len(ratings_df))

# Step 5: Train and Evaluate the Model
metrics_list = []

for fold, (train_index, test_index) in enumerate(kf.split(ratings_indices), 1):
    print(f"\nProcessing Fold {fold}/{n_splits}")
    
    # Split the ratings data into training and testing sets
    train_ratings = ratings_df.iloc[train_index]
    test_ratings = ratings_df.iloc[test_index]
    
    # Get unique user IDs from the test set
    user_ids_test = test_ratings['UserID'].unique()
    
    # Initialize the recommender
    recommender = ContentBasedRecommender(
        n_recommendations=10,
        feature_weights=None,
        distance_metric='cosine'
    )
    
    # Fit the model
    recommender.fit(item_features_df=wines_df, user_ratings_df=train_ratings)
    
    # Evaluate the model
    metrics = recommender.evaluate(user_ids=user_ids_test, ground_truth=test_ratings, top_n=10)
    metrics['Fold'] = fold
    metrics_list.append(metrics)
    
    print(f"Metrics for Fold {fold}:")
    for metric_name, value in metrics.items():
        if metric_name != 'Fold':
            print(f"{metric_name}: {value:.4f}")
    print('-' * 40)

# Step 6: Aggregate and Display the Results
metrics_df = pd.DataFrame(metrics_list)
average_metrics = metrics_df.mean(numeric_only=True)

print("\nAverage Metrics Across All Folds:")
for metric_name, value in average_metrics.items():
    print(f"{metric_name}: {value:.4f}")



Processing Fold 1/5
Metrics for Fold 1:
Precision@10: 0.0161
Recall@10: 0.0561
F1@10: 0.0236
MAP@10: 0.0180
MRR@10: 0.0469
NDCG@10: 0.0695
----------------------------------------

Processing Fold 2/5
Metrics for Fold 2:
Precision@10: 0.0150
Recall@10: 0.0511
F1@10: 0.0218
MAP@10: 0.0162
MRR@10: 0.0417
NDCG@10: 0.0632
----------------------------------------

Processing Fold 3/5
Metrics for Fold 3:
Precision@10: 0.0155
Recall@10: 0.0536
F1@10: 0.0226
MAP@10: 0.0171
MRR@10: 0.0445
NDCG@10: 0.0664
----------------------------------------

Processing Fold 4/5
Metrics for Fold 4:
Precision@10: 0.0153
Recall@10: 0.0533
F1@10: 0.0225
MAP@10: 0.0178
MRR@10: 0.0459
NDCG@10: 0.0674
----------------------------------------

Processing Fold 5/5
Metrics for Fold 5:
Precision@10: 0.0154
Recall@10: 0.0536
F1@10: 0.0225
MAP@10: 0.0176
MRR@10: 0.0450
NDCG@10: 0.0658
----------------------------------------

Average Metrics Across All Folds:
Precision@10: 0.0155
Recall@10: 0.0535
F1@10: 0.0226
MAP@10: